# Code visualizes Larmor precession for neutron spin under magnetic field

In [ ]:
import numpy as np
from qutip import *
import pandas as pd
import os
import subprocess
from IPython import display
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from scipy.constants import physical_constants

In [ ]:

def folder(name="lp_tmp"):
    """
    Checks if a folder with the given name exists. If it does not, the folder is created.
    Without the folder, images won't be plotted!!!
    """
    if not os.path.exists(name):
        os.makedirs(name)

def create_video(path, name):
    command = f'ffmpeg -framerate 90 -i {path}/frame_%d.png -c:v libx264 -pix_fmt yuv420p {name}.mp4'
    subprocess.run(command, shell=True, check=True)

def create_gif(path, name):
    n = 1
    while os.path.exists(os.path.join(path, f'output{n}.gif')):
        n += 1
    command = f'ffmpeg -framerate 30 -i {path}/frame_%d.png {name}.gif'
    subprocess.run(command, shell=True, check=True)

def display_gif(fn):

    return display.HTML('<img src="{}">'.format(fn))

def display_video(video_path):
    return display.Video(video_path)


In [ ]:
gyr_n = physical_constants["neutron gyromag. ratio"][0] #γ= 1.825*10^8 s⁻¹T⁻¹ 
B_0 = 0.0002 #T

In [ ]:
psi0 = (2.0*basis(2, 0) + basis(2, 1)).unit() # |Ψ(0)>

H = gyr_n*B_0*sigmaz()
times = np.linspace(0, (np.pi)/(gyr_n*B_0), 210)

result = sesolve(H, psi0, times, [])

In [ ]:
cmap = plt.get_cmap('seismic')
num_of_points = len(times)
colors = [cmap(i/num_of_points) for i in range(num_of_points)]

b = Bloch()

b.zlabel = ['|↑⟩', '|↓⟩']
b.point_marker = ['o']  # Set all point markers to circles
b.point_size = [20]
b.vector_color = ['green']
b.point_color = colors

n = 0
for i in range(len(result.states)):
    b.add_states(result.states[i], kind='vector')
    b.save(f'lp_tmp/frame_{n}.png', format='png')
    b.add_states(result.states[i], kind='point')
    b.save(f'lp_tmp/frame_{n+1}.png', format='png')
    b.clear()
    for j in range(i+1):
        b.add_states(result.states[j], kind='point')
    n+=2

In [ ]:
create_gif('lp_tmp', 'lp_gif')